In [41]:
import openai
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()
from serpapi import GoogleSearch




In [42]:
from openai import OpenAI
client=OpenAI(
api_key=os.environ.get("OPENAI_API_KEY")
)


In [43]:
completion=client.chat.completions.create(
    model="gpt-4o",
    messages= [
        {"role":"system", "content": "you are helpful assistant, which give details about real time of different flights."},
        {"role":"user", "content":"tell me the flight details from Jaipur to Delhi for today "}
       ],
     max_tokens=50, 
     temperature=0.8
)

In [44]:
completion

ChatCompletion(id='chatcmpl-9Y9QUnXc7jer5LSZ6Hv5pUfR8b8E8', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm really sorry, but I don't have access to real-time data. However, you can easily find the flight details from Jaipur to Delhi for today by checking the official websites of airlines, using flight tracking apps like FlightAware or FlightRadar24, or", role='assistant', function_call=None, tool_calls=None))], created=1717926414, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_319be4768e', usage=CompletionUsage(completion_tokens=50, prompt_tokens=38, total_tokens=88))

In [5]:
output=completion.choices[0].message.content
print(output)

I don't have real-time capabilities to provide current flight details. However, you can check the latest flight details from Jaipur to Delhi for today through the following methods:

1. **Airline Websites**: Major airlines such as Air India, IndiGo,


In [45]:
function_description =[
             
                    {
                "name": "get_flight_details",
                "description": "Get a real time flight details",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_location": {
                            "type": "string",
                            "description": "The departure airport, e.g. HYD, Hyderabad,",
                        },
                        "end_location": {
                            "type": "string",
                            "description": "The destination airport, e.g. MAA, Chennai",
                        },
                        
                        "date": {
                            "type": "string",
                            "description": "The format is YYYY-MM-DD. e.g. 2024-05-21"
                        }
                    },
                    "required": ["start_location", "end_location"],
                },
            },
        
     ] 

In [46]:
user_prompt="tell me the flight details from Hyderabad to Delhi on 2024-06-29"
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages= [
        {"role":"system", "content": "you are helpful assistant, which give details about real time of different flights."},
        {"role":"user", "content":user_prompt}
       ],
     function_call="auto",
     functions=function_description, 
     max_tokens=50
)

In [47]:
completion

ChatCompletion(id='chatcmpl-9Y9QgkQrCPDzKatJTFaGk481OstOV', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"start_location":"HYD","end_location":"DEL","date":"2024-06-29"}', name='get_flight_details'), tool_calls=None))], created=1717926426, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=134, total_tokens=165))

In [48]:
output=completion.choices[0].message
print(output)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"start_location":"HYD","end_location":"DEL","date":"2024-06-29"}', name='get_flight_details'), tool_calls=None)


In [49]:
def get_flight_details(start_location, end_location, date):
    params = {
        "engine": "google_flights",
        "departure_id": start_location,
        "arrival_id": end_location,
        "outbound_date": date,
        "type": "2",
        "api_key": "d306465a635303b2799c40fc08769bf89bf33a7a5d8c67c76c414a2570fe2641"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    print(results)
    return results
  



In [59]:
type(output.function_call.arguments)

str

In [57]:
start=json.loads(output.function_call.arguments).get("start_location")
end=json.loads(output.function_call.arguments).get("end_location")
date=json.loads(output.function_call.arguments).get("date")
params=json.loads(output.function_call.arguments)


In [52]:
type(params)

dict

In [53]:
print(start)
print(end)
print(params)

HYD
DEL
{'start_location': 'HYD', 'end_location': 'DEL', 'date': '2024-06-29'}


In [58]:
chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)
#print(chosen_function)
print(flight)

{'search_metadata': {'id': '66657bdc5fc493a26161b626', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/44ac0241ff1c3640/66657bdc5fc493a26161b626.json', 'created_at': '2024-06-09 09:54:36 UTC', 'processed_at': '2024-06-09 09:54:36 UTC', 'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&q=Flights+to+DEL+from+HYD+on+2024-06-29+one-way', 'raw_html_file': 'https://serpapi.com/searches/44ac0241ff1c3640/66657bdc5fc493a26161b626.html', 'prettify_html_file': 'https://serpapi.com/searches/44ac0241ff1c3640/66657bdc5fc493a26161b626.prettify', 'total_time_taken': 0.68}, 'search_parameters': {'engine': 'google_flights', 'hl': 'en', 'gl': 'us', 'type': '2', 'departure_id': 'HYD', 'arrival_id': 'DEL', 'outbound_date': '2024-06-29'}, 'best_flights': [{'flights': [{'departure_airport': {'name': 'Rajiv Gandhi International Airport', 'id': 'HYD', 'time': '2024-06-29 10:30'}, 'arrival_airport': {'name': 'Indira Gandhi International Airport (Delhi Airport)', 'id': 

In [55]:
new_name=output.function_call.name
print(new_name)

get_flight_details


In [56]:
second_completion= client.chat.completions.create(
       model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": json.dumps(flight)
},
    ],
    functions=function_description,
 )
response = second_completion.choices[0].message.content
print(response)

### Flights from Hyderabad to Delhi on 2024-06-29:

1. **Flight 1:**
   - Airline: IndiGo
   - Departure: Rajiv Gandhi International Airport, HYD at 10:30
   - Arrival: Indira Gandhi International Airport (Delhi Airport), DEL at 12:45
   - Duration: 2h 15m
   - Airplane: Airbus A321neo
   - Travel Class: Economy
   - Flight Number: 6E 6202
   - Legroom: 29 in
   - Carbon Emissions Estimate: 98 kg
   - Price: $49

2. **Flight 2:**
   - Airline: IndiGo
   - Departure: Rajiv Gandhi International Airport, HYD at 12:00
   - Arrival: Indira Gandhi International Airport (Delhi Airport), DEL at 14:20
   - Duration: 2h 20m
   - Airplane: Airbus A321neo
   - Travel Class: Economy
   - Flight Number: 6E 6031
   - Legroom: 29 in
   - Carbon Emissions Estimate: 94 kg
   - Price: $49

3. **Flight 3:**
   - Airline: Air India
   - Departure: Rajiv Gandhi International Airport, HYD at 05:30
   - Arrival: Indira Gandhi International Airport (Delhi Airport), DEL at 07:50
   - Duration: 2h 20m
   - Airpl

In [129]:
print(json.dumps(flight))

{"error": "`departure_id` (\"Hyderabad\") should either be an uppercase 3-letter code or start with \"/m\"."}
